### from : rag_cot_peactice.md

## 환경 정보

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI  # OpenAI LLM wrapper
from langchain.prompts import PromptTemplate  # PromptTemplate
from langchain.output_parsers.regex import RegexParser  # RegexParser
from langchain_core.output_parsers.json import JsonOutputParser  # JsonOutputParser

# .env 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # ex: "gpt-4o-mini"

## AnswerParser

In [2]:
docs_str = """
Doc1: RAG 파이프라인 개요 및 구성 요소
Retrieval-Augmented Generation(RAG) 파이프라인은 대규모 언어 모델(LLM)에 외부 지식을 실시간으로 결합하여 응답의 정확도와 신뢰성을 높이는 방법론입니다.
1. Retriever: 벡터 DB(예: Qdrant, Pinecone)에서 사용자 질의와 유사도가 높은 문서(top-k)를 검색
2. Document Combiner: 검색된 문서를 하나의 컨텍스트 블록으로 병합하거나 요약
3. Prompt Constructor: 합쳐진 컨텍스트와 사용자 질의를 결합해 LLM에 보낼 프롬프트 생성
4. Generator (LLM): 최종 프롬프트를 입력으로 받아 답변 생성
5. Post-processing: 불필요 정보 제거, 포맷팅, 생산된 답변 검증

Doc2: Tree of Thought(ToT) 기법 개념과 응용
Tree of Thought(ToT)는 한 번에 단일 답변을 생성하는 대신, 모델이 여러 개의 중간 추론 경로(branch)를 탐색해 최적의 사고 흐름을 선택하도록 유도하는 프롬프트 엔지니어링 기법입니다.
- 사고 노드(Node): 중간 추론 단계별로 핵심 개념이나 가설을 생성
- 분기(Branching): 서로 다른 가설 또는 접근 방식을 분리하여 평가
- 선택(Selection): 각 분기의 타당성을 비교한 뒤, 가장 적합한 사고 경로 선택
응용 사례로는 복잡한 수학 문제 풀이, 다단계 논증 생성, RAG 시스템에서 다중 문서 간 일관성 검사 등이 있으며, 특히 RAG와 결합 시 “문서 이해 → 핵심 문장 추출 → 문장 조합 → 최종 답변”의 여러 경로를 실험해 가장 신뢰도 높은 결과를 얻을 수 있습니다.

Doc3: LangChain 0.3 LCEL 스타일 Runnable 체이닝
LangChain 0.3 버전에서는 LCEL(Language Chain Execution Layer)라는 새로운 Runnable 추상화를 도입했습니다. 주요 특징은 다음과 같습니다.
1. PromptRunnable: `PromptTemplate` 객체에 따라 프롬프트를 생성하는 단계
2. LLMRunnable: 생성된 프롬프트를 LLM에 전달하여 응답을 받아오는 단계
3. ParserRunnable: LLM 출력물을 파싱(예: JSON, Regex)해 구조화된 데이터로 변환
이들 Runnable은 파이프라인 연산자(`|`)로 체이닝할 수 있어, 예를 들어:
"""

### 난이도 하 (Easy)

In [7]:
template_low = PromptTemplate(
    input_variables=['docs','query'],
    template="""
문서:
{docs}

질문: {query}

간단히 ToT 적용 이점을 설명하세요.
"""
)
llm_low = ChatOpenAI(model='gpt-4o-mini', temperature=0)
result_low = (template_low | llm_low).invoke({'docs': docs_str, 'query': 'RAG에서 ToT 적용 이점은 무엇인가요?'})
print(result_low)

content='RAG(Recovery-Augmented Generation) 시스템에 Tree of Thought(ToT) 기법을 적용하는 이점은 다음과 같습니다:\n\n1. **다양한 사고 경로 탐색**: ToT는 여러 중간 추론 경로를 탐색할 수 있도록 하여, RAG 시스템이 다양한 문서에서 얻은 정보를 보다 효과적으로 결합할 수 있게 합니다. 이를 통해 더 풍부하고 다양한 관점을 반영한 답변을 생성할 수 있습니다.\n\n2. **일관성 검사**: RAG 시스템에서 여러 문서 간의 일관성을 평가하는 데 유용합니다. ToT의 분기 및 선택 과정을 통해 서로 다른 문서에서 추출된 정보의 신뢰성을 비교하고, 가장 일관된 정보를 선택할 수 있습니다.\n\n3. **복잡한 문제 해결**: 복잡한 질문이나 다단계 논증을 요구하는 문제에 대해 ToT를 활용하면, 각 단계에서 핵심 개념을 명확히 하고, 이를 바탕으로 최적의 답변을 도출할 수 있습니다.\n\n4. **신뢰도 향상**: ToT의 구조적 접근 방식은 RAG의 결과물에 대한 신뢰도를 높이는 데 기여합니다. 여러 경로를 실험하고 평가함으로써, 최종적으로 가장 신뢰할 수 있는 답변을 선택할 수 있습니다.\n\n이러한 이점들은 RAG 시스템의 전반적인 성능과 응답의 품질을 향상시키는 데 기여합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 320, 'prompt_tokens': 610, 'total_tokens': 930, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'mo

### 난이도 중 (Medium)

In [15]:
few_shot = '''
예시)
질문: RAG에 ToT 적용 시 이점?
- 중간 사고: 문서 요약 → 핵심 개념 추출
- 최종 답변: 중간 단계를 통해 정확도를 높입니다.
'''  

few_shot = '''
예시)
질문: RAG에 ToT 적용 시 이점?
- 중간 사고: ...
- 최종 답변: ...
'''  

template_medium = PromptTemplate(
    input_variables=['few_shot','docs','query'],
    template="""
{few_shot}

문서:
{docs}

질문: {query}

아래 JSON 배열만 순수하게 출력하세요:
[
  {{"step":"중간 사고","content":"..."}},
  {{"step":"최종 답변","content":"..."}}
]
"""
)
llm_medium = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)
parser_medium = JsonOutputParser()
result_medium = (template_medium | llm_medium | parser_medium).invoke(
    # {'few_shot': few_shot, 'docs': docs_str, 'query': 'RAG ToT 적용 방법은?'}
    {'few_shot': few_shot, 'docs': docs_str, 'query': 'RAG ToT 적용 방법과 이점은 무엇인가 ?'}
)

In [16]:
print(f"type(result_medium): {type(result_medium)}, \nresult_medium: \n{result_medium}")

type(result_medium): <class 'list'>, 
result_medium: 
[{'step': '중간 사고', 'content': 'RAG와 ToT를 결합하면, RAG의 문서 검색 및 결합 과정에서 ToT의 중간 추론 경로를 활용하여 여러 문서에서 얻은 정보를 체계적으로 분석하고 평가할 수 있습니다. 이를 통해 각 문서의 핵심 내용을 추출하고, 다양한 접근 방식을 실험하여 최적의 답변을 도출할 수 있습니다.'}, {'step': '최종 답변', 'content': 'RAG에 ToT를 적용하면, 문서 이해와 핵심 문장 추출, 문장 조합의 여러 경로를 실험하여 가장 신뢰도 높은 결과를 얻을 수 있으며, 복잡한 문제 해결 및 다단계 논증 생성에서의 효과성을 높일 수 있습니다.'}]


### 난이도 상 (Hard)

In [21]:
# 중간 사고 생성 → 요약 → 최종 답변 흐름을 체이닝
# 1) 중간 사고 생성
template_first = PromptTemplate(
    input_variables=['docs','question'],
    template="""
문서:
{docs}

질문: {question}

1) 중간 사고: ToT 적용을 위한 핵심 개념 요약
2) 중간 사고: 비교 분석
3) 선택 이유 설명
"""
)
# 2) 중간 사고 요약
template_second = PromptTemplate(
    input_variables=['intermediate'],
    template="""
{intermediate}

위 중간 사고를 200자 이내로 요약하세요.
"""
)
# 3) 최종 답변 생성
template_third = PromptTemplate(
    input_variables=['summary'],
    template="""
요약:
{summary}
최종 답변을 작성하세요.
"""
)
llm_intermediate = ChatOpenAI(model='gpt-4o-mini', temperature=0)
# chain_intermediate = template_first | llm_intermediate | template_second | llm_intermediate | template_third | llm_intermediate
# chain_intermediate = template_first | llm_intermediate
chain_intermediate = template_first | llm_intermediate | template_second | llm_intermediate
response_intermediate = chain_intermediate.invoke({'docs': docs_str, 'question': 'ToT 분기 로직으로 RAG 성능 개선 방안은?'})
print(response_intermediate)

content='ToT(Thought of Tree) 기법을 RAG(Recovery-Augmented Generation)에 적용하면, 여러 문서의 정보를 깊이 분석하고 비교하여 최적의 답변을 생성할 수 있습니다. 이는 복잡한 질문에 대한 정교한 응답을 가능하게 하여 RAG의 성능을 개선합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 648, 'total_tokens': 717, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run--ffea9d19-cc32-47b5-a56e-a0f9301d1cc4-0' usage_metadata={'input_tokens': 648, 'output_tokens': 69, 'total_tokens': 717, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
